# Pushforward of matrix factorizations (tutorial)

### WARNING - you cannot save your work here

Because these notebooks are publicly accesssible, none of your edits will be saved. Even if you click the Save button, your work will be gone after you close your browser tab. (If you accidentally close your browser tab, hit `<ctrl>+<shift>+t` immediately to restore it.)

However, you can download your work by clicking File -> Download. Next time, you can upload it by clicking File -> Open and then the Upload button in the top-right corner.

### Overview

We will use the `MatrixFactorizations` package and the Julia language to compute a few compositions in the bicategory of Landau-Ginzburg models. As a concrete example, we take the matrix $Q$ from [Carqueville-Runkel][1] Thm. 7.6 that provides an orbifold equivalence between $A_{2d-1}$ and $D_{d+1}$. We compute a finite-rank representation of $Q \otimes Q^\vee$.

[1]: https://arxiv.org/abs/1210.6363v4

### Defining $Q$

We start by importing two packages and by defining the ring containing all our variables. Select the following input box and hit `<shift>+<enter>`:

In [ ]:
using PolynomialRings
using MatrixFactorizations
using GröbnerSingular
GröbnerSingular.enable()
@ring! ℚ[x,y,u,v,x₁,y₁]

If you want to type special characters such as ℚ, then type their LaTeX code (starting with `\`), and hit `<tab>`. For instance, `\bbQ<tab>`. A full list [can be found here][unicode-input].

[unicode-input]: https://docs.julialang.org/en/stable/manual/unicode-input/

In the next input box, we define the matrix factorization $Q$ for $d=2$. We take the formula from Carqueville-Runkel. The function `A_D` accepts a number `d` as in that formula. In adition, it takes two arrays defining which variables to use for the $A_{2d-1}$-potential and the $D_{d+1}$-potential.

Note that we use the symbol ⨶ for the tensor product, i.e. a tensor product with a hat. The `MatrixFactorizations` package distinguishes the following operations:

- ⊗ (`\otimes`) is the normal tensor product of matrices, i.e. the Kronecker product.
- ⨷ (`\Otimes`, note the capital `O`) is the tensor product with Koszul signs. It assumes both matrices consist of even and odd blocks.
- ⨶ (`\otimeshat`) is the composition operation in the Landau-Ginzburg category. It is equal to $A⨷1 + 1⨷B$.

Select the following input box and hit `<shift>+<enter>`:

In [ ]:
function A_D(d, A_vars, D_vars)
    _, (u,v,x,y) = polynomial_ring(A_vars..., D_vars..., basering=BigInt)
    
    f = (x^d - u^2d)÷(x - u^2)
    L = [0 x-u^2; f - y^2 0]
    R = [0 v-u*y; v+u*y 0]
    return L⨶R
end
d = 2
Q = A_D(d, [:u,:v], [:x,:y])

Let's check that it squares to the difference of the $A_{2d-1}$ and the $D_{d+1}$ potentials:

In [ ]:
Q^2

### Intermission: Inline help

If you want to try out things yourself, you may find the following resources useful:

- A [cheat sheet][cheat] for the Julia language
- An [introduction to the PolynomialRings][docs] package

There's also in-line help available by typing a question mark (`?`) in an input box, together with a search query. As an example, select the following input box and hit `<shift>+<enter>`:

[cheat]: https://cheatsheets.quantecon.org/
[docs]: https://tkluck.github.io/PolynomialRings.jl/latest/getting-started.html

In [ ]:
?unit_matrix_factorization

### Computing $hmf(D_3)$ using $Q$

We will now look at $Q \otimes Q^\vee$. The input cell below defines this matrix $M$. There's two things to note: first of all, we need to replace the $x$ and $y$ variables on one side to obtain the proper composition. We use $x_1$ and $y_1$ for that (type e.g. `x\_1<tab>`). Secondly, the resulting matrix $M$ is of infinite rank over $\mathbb{Z}[x,y,x_1,y_1]$ because of the presence of $u$ and $v$.

In [ ]:
M = Q(x=x₁,y=y₁) ⨶ dual(Q)
M^2

We will now follow [Dyckerhoff-Murfet][DM] to find a finite rank representative of the homotopy class of $M$. The following input cells are slightly technical; if you are not interested in the details you can skip forward to the 'all-in-one' paragraph below.

[DM]: https://arxiv.org/abs/1102.2957v1

The first thing we need for this construction is, for every variable $u$, a null-homotopy for some power $u^n$. It is possible to find such a homotopy because the $A_{2d-1}$-potential has a finite dimensional Jacobian. Below, we define it for $u$ and $v$, and the output shows that it works for $u^3$:

In [ ]:
λ_u = eye(Q(x=x₁,y=y₁)) ⨷ diff(dual(Q), :u) //2d; power_of_u = 2d-1
λ_v = eye(Q(x=x₁,y=y₁)) ⨷ diff(dual(Q), :v) //2; power_of_v = 1

λ_u * M + M * λ_u

We now identify $t_1 = u^3$ and $t_2 = v$. Then $\mathbb{Q}[u,v]$ is a finite-rank free module over $\mathbb{Q}[t_1,t_2]$ and so we can represent $\mathbb{Q}[u,v]$-linear maps as bigger matrices over $\mathbb{Q}[t_1, t_2]$. Below, we call this operation _inflation_. It is in these terms that we can compute the idempotent $e$ whose image is the module we are looking for.

Below, we compute the matrix factorization `MM`, which is the inflated version of $M$. We also compute $e$ and show that it is an idempotent (`e^2 == e` yields `true`). It is a morphism of the matrix factorization `MM` (`MM*e == e*MM` yields `true`).

(We're actually quite lucky that `e^2 == e`, as in general, this equation only holds up to homotopy. The all-in-one function described below takes care of that so we don't have to worry about it now.)

In [ ]:
inflated(M) = matrix_over_subring(matrix_over_subring(M, :u, power_of_u, :t₁), :v, power_of_v, :t₂)

inflated_M = inflated(M)

∇(X, t) = diff.(X, Ref(t))
At = (∇(inflated_M, :t₁) * ∇(inflated_M, :t₂) - ∇(inflated_M, :t₂) * ∇(inflated_M, :t₁))(t₁=0, t₂=0)//2

e = (inflated(λ_u) * inflated(λ_v))(t₁=0, t₂=0) * At

MM = inflated_M(t₁=0,t₂=0)
e^2 == e, MM*e == e*MM

The finite rank matrix factorization that we are looking for, is the action of `MM` on the image of `e`. We use the function `gröbner_basis` to find a reduced set of generators for this image. The function `hcat` con _cat_ enates these columns _h_ orizontally to form a matrix.

In [ ]:
image_basis = hcat(gröbner_basis(columns(e))...)

Now that we have a set of generators for the image, finding the action of `MM` on it comes down to solving an affine equation, which we can do with Gröbner basis techniques. The function `matrix_solve_affine` takes care of it for us:

In [ ]:
image_nr_columns = size(image_basis, 2)
split_matrix_size = (image_nr_columns, image_nr_columns)
MM_split = matrix_solve_affine(MM_split -> image_basis*MM_split, MM*image_basis, split_matrix_size)

We found the matrix factorization that we were looking for, but it's not very pretty. Let's have a look at the top-right block:

In [ ]:
topright(MM_split)

We can understand the structure a bit better by applying a few row/column operations. The syntax `|> RowOp(target, factor, source)` adds `factor` times the source row to the target row. Similarly, the syntax `|> RowOp(target, factor)` multiplies the target row by `factor`.

In [ ]:
MM_simple_form = MM_split |> RowOp(3, -y, 1) |> RowOp(1,2) |> RowSwap(2,3) |> ColSwap(6,7)
topright(MM_simple_form)

To finish, let's validate that the simpler form of `MM` that we found is indeed a matrix factorization linking $D_3$ to itself:

In [ ]:
MM_simple_form^2

### The all-in-one function

Because we don't want to perform the procedure above by hand every time, it is also available as a variant of the ⨶ function. For this, in addition to the two matrix factorizations, you pass the variables that you want to fuse.

For example, here's how to obtain the same thing as above:

In [ ]:
⨶( Q(x=x₁,y=y₁), dual(Q), :u, :v)

Let's apply that to a few more cases. Here's the composition of two unit factorizations of $x^3$:

In [ ]:
A = unit_matrix_factorization(x^3, [:x], [:y])
B = unit_matrix_factorization(y^3, [:y], [:u])

⨶(B, A, :y)

As it should, this yields the unit matrix factorization.

Let's apply a unit matrix factorization of the $D_3$-potential to $Q$ and see what that gives:

In [ ]:
C = unit_matrix_factorization(x^2 - x*y^2, [:x, :y], [:x₁, :y₁])
⨶(C, Q, :x, :y)

It is interesting to compare this to $Q$:

In [ ]:
Q

As we can see, it is not the same matrix. Of course, there's no need for it to be; it's only supposed to be homotopy equivalent . So we'd like to see that these two results are homotopy equivalent.

 ---- To do ----